In [1]:
from IPython.display import set_matplotlib_formats
from jax.scipy.special import logsumexp
import matplotlib.pyplot as plt
from jax import random, vmap
import jax.numpy as jnp
import jax
import seaborn as sns
import pandas as pd
import numpy as np
import time
import os

import numpyro
from numpyro.diagnostics import hpdi
import numpyro.distributions as dist
from numpyro.infer import MCMC, NUTS
from numpyro import handlers

plt.style.use('bmh')
if "NUMPYRO_SPHINXBUILD" in os.environ:
    set_matplotlib_formats('svg')

assert numpyro.__version__.startswith('0.6.0')
#numpyro.set_platform("gpu")

numpyro.set_platform("cpu")
numpyro.set_host_device_count(5)

In [2]:
def f_test(ell1, ell2):
    print('compiling')
    ell = np.minimum(ell1, ell2)
    return 2*jnp.arange(ell)


_f_test = jax.jit(f_test, static_argnums=(0, 1))

In [3]:
for ell in np.arange(5, 10):
#    m_arr = jnp.arange(ell)
    print(ell)
    print(_f_test(ell, ell+2))
    
for ell in np.arange(5, 15):
#    m_arr = jnp.arange(ell)
    print(f'executing {ell}')
    print(_f_test(ell, ell+1))

5
compiling
[0 2 4 6 8]
6
compiling
[ 0  2  4  6  8 10]
7
compiling
[ 0  2  4  6  8 10 12]
8
compiling
[ 0  2  4  6  8 10 12 14]
9
compiling
[ 0  2  4  6  8 10 12 14 16]
executing 5
compiling
[0 2 4 6 8]
executing 6
compiling
[ 0  2  4  6  8 10]
executing 7
compiling
[ 0  2  4  6  8 10 12]
executing 8
compiling
[ 0  2  4  6  8 10 12 14]
executing 9
compiling
[ 0  2  4  6  8 10 12 14 16]
executing 10
compiling
[ 0  2  4  6  8 10 12 14 16 18]
executing 11
compiling
[ 0  2  4  6  8 10 12 14 16 18 20]
executing 12
compiling
[ 0  2  4  6  8 10 12 14 16 18 20 22]
executing 13
compiling
[ 0  2  4  6  8 10 12 14 16 18 20 22 24]
executing 14
compiling
[ 0  2  4  6  8 10 12 14 16 18 20 22 24 26]


In [4]:
import jax.numpy as np
import jax
from jax import jit
from jax.tree_util import register_pytree_node
from jax.tree_util import tree_flatten, tree_unflatten

class test_class():
    def __init__(self, ell1, ell2):
        self.ell1 = ell1
        self.ell2 = ell2

    def create_m_arr(self, ell2):
        print('Compiling ', self.ell1, ell2)
        m_arr = jnp.arange(-ell2, ell2+1) + self.ell1
        return m_arr
    
    def create_m2_arr(self):
        print('Compiling ', self.ell1)
        m_arr = jnp.arange(-self.ell1, self.ell2+1)
        return m_arr    
    
    def update_ell1(self):
        print('Compiling ', self.ell1, self.ell2)
        return test_class(self.ell1+1, self.ell2)
    
    def update_ell2(self):
        print('Compiling ', self.ell1, self.ell2)
        return test_class(self.ell1, self.ell2+1)

'''
class RegisteredTest(test_class):
    def __repr__(self):
        return "RegisteredTest(ell={})".format(self.ell1)

def special_flatten(v):
    """Specifies a flattening recipe.

    Params:
        v: the value of registered type to flatten.
    Returns:
        a pair of an iterable with the children to be flattened recursively,
        and some opaque auxiliary data to pass back to the unflattening recipe.
        The auxiliary data is stored in the treedef for use during unflattening.
        The auxiliary data could be used, e.g., for dictionary keys.
    """
    children = (v.ell1, )
    aux_data = None
    return (children, aux_data)

def special_unflatten(aux_data, children):
    """Specifies an unflattening recipe.

    Params:
        aux_data: the opaque data that was specified during flattening of the
          current treedef.
    children: the unflattened children

    Returns:
        a re-constructed object of the registered type, using the specified
        children and auxiliary data.
    """
    return RegisteredTest(*children)

# Global registration
register_pytree_node(
    RegisteredTest,
    special_flatten,    # tell JAX what are the children nodes
    special_unflatten   # tell JAX how to pack back into a RegisteredTest
)
'''

# creating instance of the class                                             
try_class = test_class(5, 12)

# jitting function of the class                                              
# _create_m_arr = jax.jit(try_class.create_m_arr, static_argnums=(0,))
_create_m_arr = jax.jit(try_class.create_m_arr, static_argnums=(0, ))
_create_m2_arr = jax.jit(try_class.create_m2_arr)
print('asdf')

for ell in range(3,10):
    __ = _create_m_arr(ell).block_until_ready()
    print('Executing ', try_class.ell1, __)

print('--------Partial compilation, partial execution-------')
for ell in range(5,12):
    try_class.ell = ell
    #__ = jax.jit(try_class.create_m_arr)(ell)
    __ = _create_m_arr(ell).block_until_ready()
    print('Executing ', try_class.ell1, __)
    
print("---------Only execution after this-------")
for ell in range(4, 8):
    try_class.ell = ell
    #__ = jax.jit(try_class.create_m_arr)(ell)
    __ = _create_m_arr(ell).block_until_ready()
    print('Executing ', try_class.ell1, __)

asdf
Compiling  5 3
Executing  5 [2 3 4 5 6 7 8]
Compiling  5 4
Executing  5 [1 2 3 4 5 6 7 8 9]
Compiling  5 5
Executing  5 [ 0  1  2  3  4  5  6  7  8  9 10]
Compiling  5 6
Executing  5 [-1  0  1  2  3  4  5  6  7  8  9 10 11]
Compiling  5 7
Executing  5 [-2 -1  0  1  2  3  4  5  6  7  8  9 10 11 12]
Compiling  5 8
Executing  5 [-3 -2 -1  0  1  2  3  4  5  6  7  8  9 10 11 12 13]
Compiling  5 9
Executing  5 [-4 -3 -2 -1  0  1  2  3  4  5  6  7  8  9 10 11 12 13 14]
--------Partial compilation, partial execution-------
Executing  5 [ 0  1  2  3  4  5  6  7  8  9 10]
Executing  5 [-1  0  1  2  3  4  5  6  7  8  9 10 11]
Executing  5 [-2 -1  0  1  2  3  4  5  6  7  8  9 10 11 12]
Executing  5 [-3 -2 -1  0  1  2  3  4  5  6  7  8  9 10 11 12 13]
Executing  5 [-4 -3 -2 -1  0  1  2  3  4  5  6  7  8  9 10 11 12 13 14]
Compiling  5 10
Executing  5 [-5 -4 -3 -2 -1  0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]
Compiling  5 11
Executing  5 [-6 -5 -4 -3 -2 -1  0  1  2  3  4  5  6  7  8  9 10

In [14]:
class World:
    def __init__(self, p, v):
        self.p = p
        self.v = v

    def step(self, i, dt):
        a = - 9.8
        self.v += a * dt
        self.p += self.v *dt
        return self.v

def run(init_p, init_v):
    print('compiling')
    world = World(init_p, init_v)
    jax.lax.fori_loop(0, 1000, world.step,
                     0.01)
    return world.p, world.v

_run = jax.jit(run)

t1 = time.time()
out = _run(np.array([0, 0]), np.array([1, 1]))
t2 = time.time()
print(f'Time taken = {(t2-t1):.3f} seconds')
print(out)

t1 = time.time()
out = _run(np.array([30000.0, 2.0]), np.array([5., 1000.]))
t2 = time.time()
print(f'Time taken = {(t2-t1):.3f} seconds')
print(out)

t1 = time.time()
out = _run(np.array([301200.0, 2.0]), np.array([1234., 1000.]))
t2 = time.time()
print(f'Time taken = {(t2-t1):.3f} seconds')
print(out)

compiling


UnexpectedTracerError: Encountered an unexpected tracer. Perhaps this tracer escaped through global state from a previously traced function.
The functions being transformed should not save traced values to global state.
The tracer that caused this error was created on line <ipython-input-14-b0095e59c934>:8 (step).
When the tracer was created, the final 5 stack frames (most recent last) excluding JAX-internal frames were:
/mnt/disk2/samarth/anaconda3/envs/jax-gpu/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3357 (run_ast_nodes)
/mnt/disk2/samarth/anaconda3/envs/jax-gpu/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3437 (run_code)
<ipython-input-14-b0095e59c934>:22 (<module>)
<ipython-input-14-b0095e59c934>:15 (run)
<ipython-input-14-b0095e59c934>:8 (step)
The function being traced when the tracer leaked was while_body_fun at /mnt/disk2/samarth/anaconda3/envs/jax-gpu/lib/python3.8/site-packages/jax/_src/lax/control_flow.py:129.
To catch the leak earlier, tr

In [6]:
t1 = time.time()
out = _run(np.array([30000.0, 2.0]), np.array([5., 1000.]))
t2 = time.time()
print(f'Time taken = {(t2-t1):.6e} seconds')
print(out)

t1 = time.time()
out = _run(np.array([30000.0, 2.0]), np.array([5., 1000.]))
t2 = time.time()
print(f'Time taken = {(t2-t1):.6e} seconds')
print(out)

Time taken = 6.806850e-04 seconds
(Buffer([29559.506,  9511.512], dtype=float32), Buffer([-93., 902.], dtype=float32))
Time taken = 6.759167e-04 seconds
(Buffer([29559.506,  9511.512], dtype=float32), Buffer([-93., 902.], dtype=float32))


In [9]:
import numpy as np

In [10]:
class World:
    def __init__(self, p, v):
        self.p = p
        self.v = v

    def step(self, dt):
        a = - 9.8
        self.v += a * dt
        self.p += self.v *dt
        
def run(init_p, init_v):
    world = World(init_p, init_v)
    for i in range(1000):
        world.step(0.01)
    return world.p, world.v

t1 = time.time()
p, v = run(np.array([30000.0, 2.0]), np.array([5., 1000.]))
t2 = time.time()
print(f'Time taken = {(t2-t1):.6e} seconds')

Time taken = 8.453369e-03 seconds


In [11]:
8.453369e-03/6.806850e-04

12.418914769680542